# ReadtheDocs Retrieval Augmented Generation (RAG) using Zilliz Free Tier

In this notebook, we are going to use Milvus documentation pages to create a chatbot about our product.  The chatbot is going to follow RAG steps to retrieve chunks of data using Semantic Vector Search, then the Question + Context will be fed as a Prompt to a LLM to generate an answer.

Many RAG demos use OpenAI for the Embedding Model and ChatGPT for the Generative AI model.  **In this notebook, we will demo a fully open source RAG stack.**

Using open-source Q&A with retrieval saves money since we make free calls to our own data almost all the time - retrieval, evaluation, and development iterations.  We only make a paid call to OpenAI once for the final chat generation step. 

<div>
<img src="../images/rag_image.png" width="80%"/>
</div>

Let's get started!

In [1]:
# For colab install these libraries in this order:
# !python -m pip install torch transformers sentence-transformers langchain
# !python -m pip install -U pymilvus 'pymilvus[model]'
# !python -m pip install unstructured openai tqdm numpy ipykernel 

In [2]:
# Import common libraries.
import sys, os, time, pprint

## Download Milvus documentation.

The data we’ll use is our own product documentation web pages.  ReadTheDocs is an open-source free software documentation hosting platform, where documentation is written with the Sphinx document generator.

The code block below downloads the web pages into a local directory called `rtdocs`.  

I've already uploaded the `rtdocs` data folder to github, so you should see it if you cloned my repo.

In [3]:
# UNCOMMENT TO DOWNLOAD THE DOCS.

# # !pip install -U langchain
# from langchain_community.document_loaders import RecursiveUrlLoader

# DOCS_PAGE="https://milvus.io/docs/"

# loader = RecursiveUrlLoader(DOCS_PAGE)
# docs = loader.load()

# num_documents = len(docs)
# print(f"loaded {num_documents} documents")

In [4]:
# UNCOMMENT TO READ THE DOCS FROM A LOCAL DIRECTORY.

# Read docs into LangChain
# !pip install -U langchain
# !pip install unstructured
from langchain.document_loaders import DirectoryLoader

# Load HTML files from a local directory
path = "rtdocs/"
loader = DirectoryLoader(path, glob='*.html')
docs = loader.load()

num_documents = len(docs)
print(f"loaded {num_documents} documents")

loaded 22 documents


## Start up Milvus Lite server  <a class="anchor" id="milvus_lite"></a>

Milvus Lite is a light Python server that can run locally.  It's ideal for getting started with Milvus, running on a laptop, in a Jupyter notebook, or on Colab. 

⛔️ Please note Milvus Lite is only meant for demos, not for production workloads.

- [github](https://github.com/milvus-io/milvus-lite)
- [documentation](https://milvus.io/docs/quickstart.md)

In [5]:
# STEP 1. CONNECT A CLIENT TO LIGHT MILVUS PYTHON SERVER.

# !python -m pip install -U pymilvus
import pymilvus
print(f"pymilvus:{pymilvus.__version__}")

# Connect a client to the Milvus Lite server.
from pymilvus import MilvusClient
mc = MilvusClient("milvus_demo.db")

pymilvus:2.4.3


## Optionally, start up Milvus running in local Docker (or Zilliz free tier)

>⛔️ Make sure you pip install the correct version of pymilvus and server yml file.  **Versions (major and minor) should all match**.

1. [Install Docker](https://docs.docker.com/get-docker/)
2. Start your Docker Desktop
3. Download the latest [docker-compose.yml](https://milvus.io/docs/install_standalone-docker.md#Download-the-YAML-file) (or run the wget command, replacing version to what you are using)
> wget https://github.com/milvus-io/milvus/releases/download/v2.4.0-rc.1/milvus-standalone-docker-compose.yml -O docker-compose.yml
4. From your terminal:  
   - cd into directory where you saved the .yml file (usualy same dir as this notebook)
   - docker compose up -d
   - verify (either in terminal or on Docker Desktop) the containers are running
5. From your code (see notebook code below):
   - Import milvus
   - Connect to the local milvus server

In [6]:
# # CONNECT TO MILVUS STANDALONE DOCKER.

# import pymilvus, time
# from pymilvus import (connections, MilvusClient, utility)
# print(f"Pymilvus: {pymilvus.__version__}")

# ####################################################################################################
# # Connect to local server running in Docker container.
# # Download the latest .yaml file: https://milvus.io/docs/install_standalone-docker.md
# # Or, download directly from milvus github (replace with desired version):
# # !wget https://github.com/milvus-io/milvus/releases/download/v2.4.0-rc.1/milvus-standalone-docker-compose.yml -O docker-compose.yml
# ####################################################################################################

# # Start Milvus standalone on docker, running quietly in the background.
# # !docker compose up -d

# # # Verify which local port the Milvus server is listening on
# # !docker ps -a #19530/tcp

# # Connect to the local server.
# connection = connections.connect(
#   alias="default", 
#   host='localhost', # or '0.0.0.0' or 'localhost'
#   port='19530'
# )

# # Get server version.
# print(utility.get_server_version())

# # Use no-schema Milvus client uses flexible json key:value format.
# mc = MilvusClient(connections=connection)

# Optionally, use Zilliz free tier cluster
To use fully-managed Milvus on [Ziliz Cloud free trial](https://cloud.zilliz.com/login).  
  1. Choose the default "Starter" option when you provision > Create collection > Give it a name > Create cluster and collection.  
  2. On the Cluster main page, copy your `API Key` and store it locally in a .env variable.  See note below how to do that.
  3. Also on the Cluster main page, copy the `Public Endpoint URI`.

💡 Note: To keep your tokens private, best practice is to use an **env variable**.  See [how to save api key in env variable](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety). <br>

👉🏼 In Jupyter, you need a .env file (in same dir as notebooks) containing lines like this:
- ZILLIZ_API_KEY=f370c...
- OPENAI_API_KEY=sk-H...
- VARIABLE_NAME=value...

In [7]:
# # CONNECT TO ZILLIZ CLOUD

# # !pip install pymilvus==2.3.7 #python sdk for milvus
# import os
# import pymilvus
# print(f"pymilvus version: {pymilvus.__version__}")
# from pymilvus import connections, utility
# TOKEN = os.getenv("ZILLIZ_API_KEY")

# # Connect to Zilliz cloud using endpoint URI and API key TOKEN.
# # TODO change this.
# CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
# CLUSTER_ENDPOINT="https://in03-48a5b11fae525c9.api.gcp-us-west1.zillizcloud.com:443"
# connections.connect(
#   alias='default',
#   #  Public endpoint obtained from Zilliz Cloud
#   uri=CLUSTER_ENDPOINT,
#   # API key or a colon-separated cluster username and password
#   token=TOKEN,
# )

# # Use no-schema Milvus client uses flexible json key:value format.
# # https://milvus.io/docs/using_milvusclient.md
# mc = MilvusClient(
#     uri=CLUSTER_ENDPOINT,
#     # API key or a colon-separated cluster username and password
#     token=TOKEN)

# # Check if the server is ready and get colleciton name.
# print(f"Type of server: {utility.get_server_version()}")

## Load the Embedding Model checkpoint and use it to create vector embeddings

#### What are Embeddings?

Check out [this blog](https://zilliz.com/glossary/vector-embeddings) for an introduction to embeddings.  

An excellent place to start is by selecting an embedding model from the [HuggingFace MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard), sorted descending by the "Retrieval Average'' column since this task is most relevant to RAG. Then, choose the smallest, highest-ranking embedding model. But, Beware!! some models listed are overfit to the training data, so they won't perform on your data as promised.  

Milvus (and Zilliz) only supports tested embedding models that are not overfit.

#### In this notebook, we will use the **open-source BGE-M3** which supports: 
- over 100 languages
- context lengths of up to 8192
- multiple embedding inferences such as dense (semantic), sparse (lexical), and multi-vector Colbert reranking. 

BGE-M3 holds the distinction of being the first embedding model to offer support for all three retrieval methods, achieving state-of-the-art performance on multi-lingual (MIRACL) and cross-lingual (MKQA) benchmark tests.  [Paper](https://arxiv.org/abs/2402.03216), [HuggingFace](https://huggingface.co/BAAI/bge-m3)

**[Milvus](https://github.com/milvus-io/milvus)**, the world's first Open Source Vector Database, plays a vital role in semantic search with scaleable, efficient storage and search for GenerativeAI workflows. Its advanced functionalities include metadata filtering and hybrid search.  Since version 2.4, Milvus has built-in support for BGE M3.



<div>
<img src="../../images/bge_m3.png" width="80%"/>
</div>

In [8]:
# STEP 2. DOWNLOAD AN OPEN SOURCE EMBEDDING MODEL.
# https://huggingface.co/BAAI/bge-m3

from pymilvus.model.hybrid import BGEM3EmbeddingFunction
import torch

# Initialize torch settings
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# Initialize a Milvus built-in sparse-dense-reranking encoder.
embedding_model = BGEM3EmbeddingFunction(use_fp16=False, device=DEVICE)
EMBEDDING_DIM = embedding_model.dim['dense']
print(f"dense_dim: {EMBEDDING_DIM}")
print(f"sparse_dim: {embedding_model.dim['sparse']}")
print(f"colbert_dim: {embedding_model.dim['colbert_vecs']}")

device: cpu


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

dense_dim: 1024
sparse_dim: 250002
colbert_dim: 1024


## Create a Milvus collection

You can think of a collection in Milvus like a "table" in SQL databases.  The **collection** will contain the 
- **Schema** (or [no-schema Milvus client](https://milvus.io/docs/using_milvusclient.md)).  
💡 You'll need the vector `EMBEDDING_DIM` parameter from your embedding model.
Typical values are:
   - 1024 for sbert embedding models
   - 1536 for ada-002 OpenAI embedding models
- **Vector index** for efficient vector search
- **Vector distance metric** for measuring nearest neighbor vectors
- **Consistency level**
In Milvus, transactional consistency is possible; however, according to the [CAP theorem](https://en.wikipedia.org/wiki/CAP_theorem), some latency must be sacrificed. 💡 Searching movie reviews is not mission-critical, so [`eventually`](https://milvus.io/docs/consistency.md) consistent is fine here.

## Add a Vector Index

The vector index determines the vector **search algorithm** used to find the closest vectors in your data to the query a user submits.  

Most vector indexes use different sets of parameters depending on whether the database is:
- **inserting vectors** (creation mode) - vs - 
- **searching vectors** (search mode) 

Scroll down the [docs page](https://milvus.io/docs/index.md) to see a table listing different vector indexes available on Milvus.  For example:
- FLAT - deterministic exhaustive search
- IVF_FLAT or IVF_SQ8 - Hash index (stochastic approximate search)
- HNSW - Graph index (stochastic approximate search)
- AUTOINDEX - OSS or [Zilliz cloud](https://docs.zilliz.com/docs/autoindex-explained) automatic index based on type of GPU, size of data.

Besides a search algorithm, we also need to specify a **distance metric**, that is, a definition of what is considered "close" in vector space.  In the cell below, the [`HNSW`](https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md) search index is chosen.  Its possible distance metrics are one of:
- L2 - L2-norm
- IP - Dot-product
- COSINE - Angular distance

💡 Most use cases work better with normalized embeddings, in which case L2 is useless (every vector has length=1) and IP and COSINE are the same.  Only choose L2 if you plan to keep your embeddings unnormalized.

In [9]:
# STEP 3. CREATE A MILVUS COLLECTION AND DEFINE THE DATABASE INDEX.

COLLECTION_NAME = "MilvusDocs"

# Check if collection already exists, if so drop it.
if mc.has_collection(COLLECTION_NAME):
    mc.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# Create a collection with flexible schema and AUTOINDEX.
# Uses Milvus AUTOINDEX, which defaults to HNSW.
mc.create_collection(COLLECTION_NAME, 
        EMBEDDING_DIM,
        consistency_level="Eventually", 
        auto_id=True,  
        overwrite=True,
    )
print(f"Successfully created collection: `{COLLECTION_NAME}`")

Successfully created collection: `MilvusDocs`


## Chunking

Before embedding, it is necessary to decide your chunk strategy, chunk size, and chunk overlap.  In this demo, I will use:
- **Strategy** = Use markdown header hierarchies.  Keep markdown sections together unless they are too long.
- **Chunk size** = Use the embedding model's parameter `MAX_SEQ_LENGTH`
- **Overlap** = Rule-of-thumb 10-15%
- **Function** = 
  - Langchain's `HTMLHeaderTextSplitter` to split markdown sections.
  - Langchain's `RecursiveCharacterTextSplitter` to split up long reviews recursively.


Notice below, each chunk is grounded with the document source page.  <br>
In addition, header titles are kept together with the chunk of markdown text.

In [10]:
# # STEP 4. PREPARE DATA: CHUNK AND EMBED

# !python -m pip install lxml
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter

# Define chunk size 512 and overlap 10% chunk_size.
chunk_size = 512
chunk_overlap = round(chunk_size * 0.10, 0)
print(f"chunk_size: {chunk_size}, chunk_overlap: {chunk_overlap}")

# Create an instance of the RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
    separators=["\n", "\n\n", "<br /><br />"],
)

# Define the headers to split on for the HTMLHeaderTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]
# Create an instance of the HTMLHeaderTextSplitter
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Split the HTML text using the HTMLHeaderTextSplitter.
start_time = time.time()
html_header_splits = []
for doc in docs:
    splits = html_splitter.split_text(doc.page_content)
    for split in splits:
        # Add the source URL and header values to the metadata
        metadata = {}
        new_text = split.page_content
        for header_name, metadata_header_name in headers_to_split_on:
            # Handle exception if h1 does not exist.
            try:
                header_value = new_text.split("¶ ")[0].strip()[:50]
                metadata[header_name] = header_value
            except:
                break
            # Handle exception if h2 does not exist.
            try:
                new_text = new_text.split("¶ ")[1].strip()[:50]
                metadata[header_name] = new_text
            except:
                break
        split.metadata = {
            **metadata,
            "source": doc.metadata["source"]
        }
    html_header_splits.extend(splits)

# Split the documents further into smaller, recursive chunks.
chunks = child_splitter.split_documents(html_header_splits)

# Add the headers to the chunk except for the 0th chunk per header.
for chunk in chunks:
    if chunk.page_content[:50] != chunk.metadata['h1']:
        metadata_str = ' '.join(f'{v}' for k, v in split.metadata.items() if k != 'source')
        chunk.page_content = metadata_str + ' ' + chunk.page_content

end_time = time.time()
print(f"chunking time: {end_time - start_time}")
print(f"docs: {len(docs)}, split into chunks: {len(chunks)}")
print(f"type: list of {type(chunks[0])}") 

# Inspect a chunk.
print()
print("Looking at a sample chunk...")
print(chunks[0].page_content[:100])
print(chunks[0].metadata)

# # TODO - Uncomment to print child splits with their associated header metadata for debugging.
# print()
# for child in chunks:
#     print(f"Content: {child.page_content}")
#     print(f"Metadata: {child.metadata}")
#     print()

# Clean up the metadata urls
for doc in chunks:
    new_url = doc.metadata["source"]
    new_url = new_url.replace("rtdocs", "https://milvus.io/docs")
    new_url = new_url.replace(".html", ".md")
    doc.metadata.update({"source": new_url})

print(chunks[0].page_content[:100])
print(chunks[0].metadata)

chunk_size: 512, chunk_overlap: 51.0
chunking time: 0.03227996826171875
docs: 22, split into chunks: 304
type: list of <class 'langchain_core.documents.base.Document'>

Looking at a sample chunk...
Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo
{'h1': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo', 'source': 'rtdocs/quickstart.html'}
Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo
{'h1': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo', 'source': 'https://milvus.io/docs/quickstart.md'}


Use the built-in Milvus BGE M3 embedding functions.  The output will be 2 vectors:
- `embeddings['dense'][i]` is a list of numpy arrays, one per chunk. Milvus supports more than 1 dense embedding vector if desired, so i is the ith dense embedding vector.
- `embeddings['sparse'][:, [i]]` is a scipy sparse matrix where each column represents a chunk.

In [11]:
# STEP 5. TRANSFORM CHUNKS INTO VECTORS USING EMBEDDING MODEL INFERENCE.

# BGEM3EmbeddingFunction input is docs as a list of strings.
list_of_strings = [doc.page_content for doc in chunks if hasattr(doc, 'page_content')]

# Embedding inference using the Milvus built-in sparse-dense-reranking encoder.
start_time = time.time()
embeddings = embedding_model(list_of_strings)
end_time = time.time()

print(f"Embedding time for {len(list_of_strings)} chunks: ", end="")
print(f"{round(end_time - start_time, 2)} seconds")

# Inference Embeddings: 100%|██████████| 19/19 [00:35<00:00,  1.86s/it]
# Embedding time for 304 chunks: 35.74 seconds

Inference Embeddings: 100%|██████████| 19/19 [00:52<00:00,  2.76s/it]

Embedding time for 304 chunks: 52.59 seconds


## Insert data into Milvus

For each original text chunk, we'll write the sextuplet (`chunk, h1, h2, source, dense_vector, sparse_vector`) into the database.

<div>
<img src="../images/db_insert.png" width="80%"/>
</div>

**The Milvus Client wrapper can only handle loading data from a list of dictionaries.**

Otherwise, in general, Milvus supports loading data from:
- pandas dataframes 
- list of dictionaries

In [12]:
# STEP 6. INSERT CHUNK LIST INTO MILVUS OR ZILLIZ.

# Create chunk_list and dict_list in a single loop
dict_list = []
for chunk, sparse, dense in zip(chunks, embeddings["sparse"], embeddings["dense"]):
    # Assemble embedding vector, original text chunk, and metadata.
    chunk_dict = {
        'chunk': chunk.page_content,
        'h1': chunk.metadata.get('h1', "")[:50],
        'h2': chunk.metadata.get('h2', "")[:50],
        'source': chunk.metadata.get('source', ""),
        # 'sparse_vector': sparse,
        # 'dense_vector': dense,
        'vector': dense
    }
    dict_list.append(chunk_dict)

# TODO - Uncomment to inspect the first chunk and its metadata.
print(len(dict_list))
print(type(dict_list[0]), len(dict_list[0]))
pprint.pprint(dict_list[0])

304
<class 'dict'> 5
{'chunk': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed '
          'Milvus FREE Search Home v2.4.x About Milvus Get '
          'StartedPrerequisitesInstall MilvusInstall SDKsQuickstart Concepts '
          'User Guide Embeddings Administration Guide Tools Integrations '
          'Example Applications FAQs API reference Quickstart This guide '
          'explains how to connect to your Milvus cluster and performs CRUD '
          'operations in minutes Before you start You have installed Milvus '
          'standalone or Milvus cluster. You have installed preferred SDKs. '
          'You can',
 'h1': 'Why Milvus Docs Tutorials Tools Blog Community Sta',
 'h2': '',
 'source': 'https://milvus.io/docs/quickstart.md',
 'vector': array([-0.01666473,  0.05284619, -0.05246121, ..., -0.01825546,
        0.03670057, -0.00945158], dtype=float32)}


In [13]:
# Insert data into the Milvus collection.
print("Start inserting entities")
start_time = time.time()
mc.insert(
    COLLECTION_NAME,
    data=dict_list,
    progress_bar=True)

end_time = time.time()
print(f"Milvus insert time for {len(dict_list)} vectors: ", end="")
print(f"{round(end_time - start_time, 2)} seconds")

Start inserting entities
Milvus insert time for 304 vectors: 0.17 seconds


## Aside - example Milvus collection API calls
https://milvus.io/docs/manage-collections.md#View-Collections

Below are some common API calls for checking a collection.
- `.num_entities`, flushes data and executes row count.
- `.describe_collection()`, gives details about the schema, index, collection.
- `.query()`, gives back selected data from the collection.

In [14]:
# Example Milvus Collection utility API calls.
# https://milvus.io/docs/manage-collections.md#View-Collections

# View collection info, incurs a call to .flush() first.
start_time = time.time()
pprint.pprint(mc.describe_collection(COLLECTION_NAME))
end_time = time.time()
print(f"timing: {round(end_time - start_time, 4)} seconds")
print()

# Count rows without incurring call to .flush().
start_time = time.time()
res = mc.query( collection_name=COLLECTION_NAME, 
               filter="", 
               output_fields = ["count(*)"], )
pprint.pprint(res)
end_time = time.time()
print(f"timing: {round(end_time - start_time, 4)} seconds")

{'aliases': [],
 'auto_id': True,
 'collection_id': 0,
 'collection_name': 'MilvusDocs',
 'consistency_level': 0,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'auto_id': True,
             'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {},
             'type': <DataType.INT64: 5>},
            {'description': '',
             'field_id': 101,
             'name': 'vector',
             'params': {'dim': 1024},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 0,
 'num_shards': 0,
 'properties': {}}
timing: 0.0011 seconds

data: ["{'count(*)': 304}"] , extra_info: {'cost': 0}
timing: 0.0028 seconds


## Ask a question about your data

So far in this demo notebook: 
1. Your custom data has been mapped into a vector embedding space
2. Those vector embeddings have been saved into a vector database

Next, you can ask a question about your custom data!

💡 In LLM vocabulary:
> **Query** is the generic term for user questions.  
A query is a list of multiple individual questions, up to maybe 1000 different questions!

> **Question** usually refers to a single user question.  
In our example below, the user question is "What is AUTOINDEX in Milvus Client?"

> **Semantic Search** = very fast search of the entire knowledge base to find the `TOP_K` documentation chunks with the closest embeddings to the user's query.

💡 The same model should always be used for consistency for all the embeddings data and the query.

In [15]:
# Define a sample question about your data.
QUESTION1 = "What do the parameters for HNSW mean?"
QUESTION2 = "What are good default values for HNSW parameters with 25K vectors dim 1024?"
QUESTION3 = "What does nlist vs nprobe mean in ivf_flat?"
QUESTION4 = "What is the default AUTOINDEX index and vector field distance metric in Milvus?"

# In case you want to ask all the questions at once.
QUERY = [QUESTION1, QUESTION2, QUESTION3, QUESTION4]

# Inspect the length of one question.
QUERY_LENGTH = len(QUESTION2)
print(f"query length: {QUERY_LENGTH}")

query length: 75


In [16]:
# SELECT A PARTICULAR QUESTION TO ASK.

SAMPLE_QUESTION = QUESTION1

## Execute a vector search

Search Milvus using [PyMilvus API](https://milvus.io/docs/search.md).

💡 By their nature, vector searches are "semantic" searches.  For example, if you were to search for "leaky faucet": 
> **Traditional Key-word Search** - either or both words "leaky", "faucet" would have to match some text in order to return a web page or link text to the document.

> **Semantic search** - results containing words "drippy" "taps" would be returned as well because these words mean the same thing even though they are different words.

In [17]:
# Import custom functions for splitting and search.
sys.path.append("../")  # Adds higher directory to python modules path.
import milvus_utilities as _utils

In [18]:
# Define metadata fields you can filter on.
OUTPUT_FIELDS = list(dict_list[0].keys())
OUTPUT_FIELDS.remove('vector')
print(f"output fields: {OUTPUT_FIELDS}")

# Define a convenience function for searching.
def mc_run_search(question, filter_expression, top_k=3):
    # Embed the question using the same encoder.
    query_embeddings = embedding_model([SAMPLE_QUESTION])['dense']

    # Run semantic vector search using your query and the vector database.
    results = mc.search(
        COLLECTION_NAME,
        data=query_embeddings, 
        # search_params=SEARCH_PARAMS,
        output_fields=OUTPUT_FIELDS, 
        # Milvus can utilize metadata in boolean expressions to filter search.
        filter=filter_expression,
        limit=top_k,
        consistency_level="Eventually"
    )

    # Assemble retrieved context and context metadata.
    # The search result is in the variable `results[0]`, which is type 
    # 'pymilvus.orm.search.SearchResult'. 
    METADATA_FIELDS = [f for f in OUTPUT_FIELDS if f != 'chunk']
    formatted_results, context, context_metadata = _utils.client_assemble_retrieved_context(
        results, metadata_fields=METADATA_FIELDS, num_shot_answers=TOP_K)
    
    return formatted_results, context, context_metadata

output fields: ['chunk', 'h1', 'h2', 'source']


In [19]:
# STEP 7. RETRIEVE ANSWERS FROM YOUR DOCUMENTS STORED IN MILVUS OR ZILLIZ.

# Metadata filters for CSV dataset.
# expression = 'film_year >= 2019'
expression = ""
TOP_K = 2

start_time = time.time()
formatted_results, contexts, context_metadata = \
    mc_run_search(SAMPLE_QUESTION, expression, TOP_K)
elapsed_time = time.time() - start_time
print(f"Milvus Client search time for {len(dict_list)} vectors: {elapsed_time} seconds")

# Inspect search result.
print(f"type: {type(formatted_results)}, count: {len(formatted_results)}")

Milvus Client search time for 304 vectors: 0.8612580299377441 seconds
type: <class 'list'>, count: 2


## Assemble and inspect the search result

The search result is in the variable `results[0]` consisting of top_k-count of objects of type `'pymilvus.client.abstract.Hits'`



In [20]:
# Loop through search results, print metadata.
sources = []
for i in range(len(contexts)):
    print(f"Retrieved result #{i+1}")
    print(f"distance = {formatted_results[i][0]}")
    pprint.pprint(f"Chunk text: {contexts[i]}")
    for key, value in context_metadata[i].items():
        if key == "source":
            sources.append(value)
        print(f"{key}: {value}")
    print()

Retrieved result #1
distance = 0.6101710200309753
('Chunk text: this value can improve recall rate at the cost of increased '
 'search time. [1, 65535] 2 HNSW HNSW (Hierarchical Navigable Small World '
 'Graph) is a graph-based indexing algorithm. It builds a multi-layer '
 'navigation structure for an image according to certain rules. In this '
 'structure, the upper layers are more sparse and the distances between nodes '
 'are farther; the lower layers are denser and the distances between nodes are '
 'closer. The search starts from the uppermost layer, finds the node closest '
 'to the target')
h1: Why Milvus Docs Tutorials Tools Blog Community Sta
h2: 
source: https://milvus.io/docs/index.md

Retrieved result #2
distance = 0.5427447557449341
('Chunk text: layer, finds the node closest to the target in this layer, and '
 'then enters the next layer to begin another search. After multiple '
 'iterations, it can quickly approach the target position. In order to improve '
 'performanc

## Use an LLM to Generate a chat response to the user's question using the Retrieved Context.

Many different generative LLMs exist these days.  Check out the lmsys [leaderboard](https://chat.lmsys.org/?leaderboard).

In this notebook, we'll try these LLMs:
- The newly released open-source Llama 3 from Meta.
- The cheapest, paid model from Anthropic Claude3 Haiku.
- The standard in its price cateogory, gpt-3.5-turbo, from Openai.

In [21]:
# STEP 8. LLM-GENERATED ANSWER TO THE QUESTION, GROUNDED BY RETRIEVED CONTEXT.

# Separate all the context together by space.
# Lance Martin, LangChain, says put best contexts at end.
contexts_combined = ' '.join(reversed(contexts))
# Separate all the sources together by comma.
source_combined = ' '.join(reversed(sources))
print(f"Length long text to summarize: {len(contexts_combined)}")

# Define temperature for the LLM and random seed.
TEMPERATURE = 0.1
TOP_P = 0.9
RANDOM_SEED = 415
MAX_TOKENS = 512
FREQUENCY_PENALTY = 2

Length long text to summarize: 1017


In [22]:
SYSTEM_PROMPT = f"""First, check if the Context below is relevant to 
the user's question.  Second if the context is not strongly relevant, 
throw away the context.  Third, if the context is strongly relevant, 
answer the question using the context; otherwise ignore the context 
and answer the question.  Be complete, clear, concise.  
Answer with fewer than 3 sentences and cite unique sources at the end.
Answer: The answer to the question.
Grounding sources: {source_combined}
Context: {contexts_combined}
"""
print(f"Length prompt: {len(SYSTEM_PROMPT)}")

Length prompt: 1534


# Try Llama3 with Ollama to generate a human-like chat response to the user's question

Follow the instructions to install ollama and pull a model.<br>
https://github.com/ollama/ollama

View details about which models are supported by ollama. <br>
https://ollama.com/library/llama3

That page says `ollama run llama3` will by default pull the latest "instruct" model, which is fine-tuned for chat/dialogue use cases.

The other kind of llama3 models are "pre-trained" base model. <br>
Example: ollama run llama3:text ollama run llama3:70b-text

**Format** `gguf` means the model runs on CPU.  gg = "Georgi Gerganov", creator of the C library model format ggml, which was recently changed to gguf.

**Quantization** (think of it like vector compaction) can lead to higher throughput at the expense of lower accuracy.  For the curious, quantization meanings can be found on: <br>
https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/tree/main.  

Below just listing the main quantization types.
- **q4_0**: Original quant method, 4-bit.
- **q4_k_m**: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
- **q5_0**: Higher accuracy, higher resource usage and slower inference.
- **q5_k_m**: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
- **q 6_k**: Uses Q8_K for all tensors
- **q8_0**: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

In [23]:
# !python -m pip install ollama
import ollama

# Verify details which model you are running.
ollama_llama3 = ollama.list()['models'][0]

# Print the model details.
keys = ['format', 'parameter_size', 'quantization_level']
print(f"MODEL:{ollama.list()['models'][0]['name']}", end=", ")
for key in keys:
    print(f"{str.upper(key)}:{ollama.list()['models'][0]['details'].get(key, 'Key not found in dictionary')}", end=", ")
print(end="\n\n")

MODEL:llama3:latest, FORMAT:gguf, PARAMETER_SIZE:8B, QUANTIZATION_LEVEL:Q4_0, 



In [24]:
# Send the question to llama 3 chat.
start_time = time.time()
response = ollama.chat(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model='llama3',
    stream=False,
    options={"temperature": TEMPERATURE, "seed": RANDOM_SEED,
             "top_p": TOP_P, 
            #  "max_tokens": MAX_TOKENS,
             "frequency_penalty": FREQUENCY_PENALTY}
)

ollama_llama3_time = time.time() - start_time
pprint.pprint(response['message']['content'].replace('\n', ' '))
print(f"ollama_llama3_time: {format(ollama_llama3_time, '.2f')} seconds")

('According to Milvus documentation [1], in Hierarchical Navigable Small World '
 'Graph (HNSW) algorithm:  * `M`: Maximum degree of nodes on each layer. This '
 'value can improve recall rate at the cost of increased search time. \t+ '
 'Range: `(2, 2048)` * No other parameters are mentioned.  These two '
 'parameters control how HNSW builds its multi-layer navigation structure and '
 'affect the trade-off between search efficiency and recall rate.  References: '
 '[1] https://milvus.io/docs/index.md')
ollama_llama3_time: 25.41 seconds


# Now try Anyscale endpoints

Install vllm and anyscale endpoints:  https://github.com/simonw/llm-anyscale-endpoints

In [25]:
# # List all the anyscale endpoint models.
# !llm models list

In [26]:
# Call Anyscale enpoint using OpenAI API.
import openai

LLM_NAME = "meta-llama/Llama-3-8b-chat-hf"

# 1. Get your API key: https://platform.openai.com/api-keys
# 2. Save your api key in env variable.
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
anyscale_client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key=os.environ.get("ANYSCALE_ENPOINT_KEY"),
)

# 3. Generate response using the OpenAI API.
start_time = time.time()
response = anyscale_client.chat.completions.create(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model=LLM_NAME,
    temperature=TEMPERATURE,
    seed=RANDOM_SEED,
    frequency_penalty=FREQUENCY_PENALTY,
    top_p=TOP_P, 
    max_tokens=MAX_TOKENS,
)
llama3_anyscale_endpoints_time = time.time() - start_time

# Print the response.
pprint.pprint(response.choices[0].message.content.replace('\n', ' '))
print(f"llama3_anyscale_endpoints_time: {format(llama3_anyscale_endpoints_time, '.2f')} seconds")

('According to the context, the parameters for HNSW (Hierarchical Navigable '
 'Small World Graph) are:  * M: Maximum degree of the node, which can be set '
 'to a value between 2 and 2048. This parameter can improve the recall rate at '
 'the cost of increased search time. * efConstruction (when building index) or '
 'ef (when searching targets): Specify a search range.  These parameters can '
 'be used to adjust the performance of the HNSW algorithm.')
llama3_anyscale_endpoints_time: 3.85 seconds


# Also try OctoAI

In [27]:
# !python -m pip install octoai
from octoai.text_gen import ChatMessage
from octoai.client import OctoAI

LLM_NAME = "meta-llama-3-8b-instruct"

octoai_client = OctoAI(
    api_key=os.environ.get("OCTOAI_TOKEN"),
)

# Generate response using OpenAI API.
start_time = time.time()
response = octoai_client.text_gen.create_chat_completion(
	messages=[
		ChatMessage(
			content=SYSTEM_PROMPT,
			role="system"
		),
		ChatMessage(
			content=SAMPLE_QUESTION,
			role="user"
		)
	],
	model=LLM_NAME,
    temperature=TEMPERATURE,
    # seed=RANDOM_SEED,
    frequency_penalty=FREQUENCY_PENALTY,
    top_p=TOP_P, 
    max_tokens=MAX_TOKENS,
)
llama3_octai_endpoints_time = time.time() - start_time

# Print the response.
pprint.pprint(response.choices[0].message.content.replace('\n', ' '))
print(f"llama3_octai_endpoints_time: {format(llama3_octai_endpoints_time, '.2f')} seconds")

('According to the context, the parameters for HNSW (Hierarchical Navigable '
 'Small World Graph) are:  * M: Maximum degree of nodes on each layer of the '
 'graph, which can be set between 2 and 2048. This value can improve recall '
 'rate at the cost of increased search time. * efConstruction (when building '
 'index): specifies a search range during index construction. * ef (when '
 'searching targets): specifies a search range during target searching.  These '
 'parameters help control how efficiently HNSW searches for nearest neighbors '
 'in an image.')
llama3_octai_endpoints_time: 1.98 seconds


# Also try Groq endpoints

In [28]:
# !python -m pip install groq
from groq import Groq

LLM_NAME = "llama3-8b-8192"

groq_client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Generate response using OpenAI API.
start_time = time.time()
response = groq_client.chat.completions.create(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model=LLM_NAME,
    temperature=TEMPERATURE,
    seed=RANDOM_SEED,
    frequency_penalty=FREQUENCY_PENALTY,
    top_p=TOP_P, 
    max_tokens=MAX_TOKENS,
)
llama3_groq_endpoints_time = time.time() - start_time

# Print the response.
pprint.pprint(response.choices[0].message.content.replace('\n', ' '))
print(f"llama3_groq_endpoints_time: {format(llama3_groq_endpoints_time, '.2f')} seconds")

('According to the context, the parameters for HNSW (Hierarchical Navigable '
 'Small World Graph) are:  * M: Maximum degree of nodes on each layer of the '
 'graph, ranging from 2 to 2048. * efConstruction (when building index): '
 'specifies the search range. * ef (when searching targets): specifies the '
 'search range.  These parameters can be adjusted to improve performance, such '
 'as increasing the recall rate at the cost of increased search time.')
llama3_groq_endpoints_time: 0.71 seconds


## Also try MistralAI's Mixtral 8x7B-Instruct-v0.1

This time ollama's version requires 48GB RAM. If you have big enough compute, run the command:
> ollama run mixtral

Since my laptop is a M2 with only 16GB RAM, I decided to **run Mixtral using Anyscale Endpoints**. Instructions to install. <br>
> https://github.com/simonw/llm-anyscale-endpoints

To get back to **Anyscale Endpoints** anytime, open the playground.<br>
https://console.anyscale.com/v2/playground

In [29]:
# Call Anyscale enpoint using OpenAI API.
import openai

LLM_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# 2. Get your API key: https://platform.openai.com/api-keys
# 3. Save your api key in env variable.
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai_client = openai.OpenAI(
    base_url = "https://api.endpoints.anyscale.com/v1",
    api_key=os.environ.get("ANYSCALE_ENPOINT_KEY"),
)

# 4. Generate response using the OpenAI API.
start_time = time.time()
response = openai_client.chat.completions.create(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model=LLM_NAME,
    temperature=TEMPERATURE,
    seed=RANDOM_SEED,
    frequency_penalty=FREQUENCY_PENALTY,
)
mixtral_anyscale_endpoints_time = time.time() - start_time

# Print the response.
pprint.pprint(response.choices[0].message.content.replace('\n', ' '))
print(f"mixtral_anyscale_endpoints_time: {format(mixtral_anyscale_endpoints_time, '.2f')} seconds")

(' The parameters for HNSW are as follows:  1. M (Maximum degree of the node): '
 'This is the maximum number of neighbors a node can have in the graph. It '
 'ranges from 2 to 2048, and increasing this value can improve recall rate but '
 'at the cost of increased search time. 2. efConstruction (used when building '
 'index) or ef (used when searching targets): These specify a search range '
 'during index building or target searching, respectively. They range from 1 '
 'to 65535, with higher values resulting in a larger search range and '
 'potentially better recall rates but also increased computation time.  '
 'Source: '
 '<https://milvus.io/docs/index-hswn_tuning_guide-020798b6a7b44e3ea9dabaaaedbeaecc.html>')
mixtral_anyscale_endpoints_time: 3.92 seconds


## Also try OpenAI

💡 Note: For use cases that need to always be factually grounded, use very low temperature values while more creative tasks can benefit from higher temperatures.

In [30]:
SYSTEM_PROMPT = f"""Use the Context below to answer the user's question. 
Be clear, factual, complete, concise.
If the answer is not in the Context, say "I don't know". 
Otherwise answer with fewer than 4 sentences and cite the grounding sources.
Context: {contexts_combined}
Answer: The answer to the question.
Grounding sources: {source_combined}
"""

In [31]:
import openai, pprint
from openai import OpenAI

# 1. Define the generation llm model to use.
# https://openai.com/blog/new-embedding-models-and-api-updates
# Customers using the pinned gpt-3.5-turbo model alias will be automatically upgraded to gpt-3.5-turbo-0125 two weeks after this model launches.
LLM_NAME = "gpt-3.5-turbo"

# 2. Get your API key: https://platform.openai.com/api-keys
# 3. Save your api key in env variable.
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# 4. Generate response using the OpenAI API.
start_time = time.time()
response = openai_client.chat.completions.create(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model=LLM_NAME,
    temperature=TEMPERATURE,
    seed=RANDOM_SEED,
    frequency_penalty=FREQUENCY_PENALTY,
)
chatgpt_35turbo_time = time.time() - start_time

# Print the question and answer along with grounding sources and citations.
print(f"Question: {SAMPLE_QUESTION}")

# 5. Print all answers in the response.
for i, choice in enumerate(response.choices, 1):
    pprint.pprint(f"Answer: {choice.message.content}")
    print("\n")
print(f"chatgpt_3.5_turbo_time: {format(chatgpt_35turbo_time, '.5f')}")

# Question1: What do the parameters for HNSW mean?
# Answer:  Looks perfect!
# Best answer:  M: maximum degree of nodes in a layer of the graph. 
# efConstruction: number of nearest neighbors to consider when connecting nodes in the graph.
# ef: number of nearest neighbors to consider when searching for similar vectors. 

# Question2: What are good default values for HNSW parameters with 25K vectors dim 1024?
# Answer: M=16, efConstruction=500, and ef=64
# Best answer:  M=16, efConstruction=32, ef=32

# Question3: what is the default distance metric used in AUTOINDEX in Milvus?
# Answer: L2 
# Trick answer:  IP inner product, not yet updated in documentation still says L2.

Question: What do the parameters for HNSW mean?
('Answer: The parameters for HNSW are as follows:\n'
 '- M: Maximum degree of the node (2, 2048), which limits the maximum number '
 'of connections a node can have in each layer.\n'
 '- efConstruction: A parameter used during index building to specify a search '
 'range, affecting recall rate and search time.\n'
 '- ef: A parameter used when searching for targets to specify a search range. '
 'It also impacts recall rate and search time. \n'
 'These parameters help control the structure and performance of the '
 'graph-based indexing algorithm HNSW.')


chatgpt_3.5_turbo_time: 2.08135


In [32]:
# Drop collection
mc.drop_collection(COLLECTION_NAME)

In [33]:
# Props to Sebastian Raschka for this handy watermark.
# !pip install watermark

%load_ext watermark
%watermark -a 'Christy Bergman' -v -p unstructured,lxml,torch,pymilvus,langchain,ollama,octoai,groq,openai --conda

Author: Christy Bergman

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

unstructured: 0.14.4
lxml        : 5.1.0
torch       : 2.3.0
pymilvus    : 2.4.3
langchain   : 0.2.2
ollama      : 0.1.8
octoai      : 1.0.2
groq        : 0.8.0
openai      : 1.14.3

conda environment: py311-unum

